# Store chunks into Vector Database using Azure Cognitive Search (ACS)

In [28]:
import os
import re
import pandas as pd
import json  
import openai  
from dotenv import load_dotenv
from tenacity import retry, wait_random_exponential, stop_after_attempt  
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient  
from azure.search.documents.indexes import SearchIndexClient  
from azure.search.documents.models import Vector  
from azure.search.documents.indexes.models import (  
    SearchIndex,  
    SearchField,  
    SearchFieldDataType,  
    SimpleField,  
    SearchableField,  
    SearchIndex,  
    SemanticConfiguration,  
    PrioritizedFields,  
    SemanticField,  
    SearchField,  
    SemanticSettings,  
    VectorSearch,  
    HnswVectorSearchAlgorithmConfiguration
)
from langchain.text_splitter import RecursiveCharacterTextSplitter
from ast import literal_eval

# Load environment variables and keys 

In [17]:
from dotenv import dotenv_values

# specify the name of the .env file name 
env_name = "../../llm.env" # change to your own .env file name
config = dotenv_values(env_name)

# Azure OpenAI
openai.api_type = config["OPENAI_API_TYPE"] #"azure"
openai.api_key = config['OPENAI_API_KEY']
openai.api_base = config['OPENAI_API_BASE']
openai.api_version = config['OPENAI_API_VERSION']

## Cog Search
cogsearch_name = config["COGSEARCH_NAME"]
index_name = config["COGSEARCH_INDEX_NAME"]
key = config["COGSEARCH_API_KEY"]
service_endpoint = "https://"+config["COGSEARCH_NAME"] + ".search.windows.net"

credential = AzureKeyCredential(key)

In [34]:
def createEmbeddings(text):
    response = openai.Embedding.create(input=text , engine=config["OPENAI_DEPLOYMENT_EMBEDDING"])
    embeddings = response['data'][0]['embedding']
    return embeddings

#### Store the embeddings in Azure Cognitive Search Vector Store

[AzureCogSearch](https://learn.microsoft.com/en-us/azure/search/search-what-is-azure-search) provides a simple interface to create a vector database, store and retrieve data using vector search. You can read more about [here](https://github.com/Azure/cognitive-search-vector-pr/tree/main) more about Vector Search.

There are two steps to store data in AzureCogSearch vector database:
- First, we create the index (or schema) of the vector database
- Second, we add the chunked documents and their embeddings to the vector datastore

In [18]:
df_chunks_embedding = pd.read_csv('AnalyzedPDF/ChunksEmbedding.csv')

In [19]:
df_chunks_embedding.head(3)
#columns should look like the following with order preserved
#Id, Chunk, PageNumber, LineNumber, DocId, Embedding

,Id,Ticker,Year,Quarter,Chunk,PageNumber,LineNumber,Embedding
0,1,MSFT,23,1,Microsoft FY23 First Quarter Earnings Conferen...,1,1,"[-0.022691456601023674, -0.028929658234119415,..."
1,2,MSFT,23,1,"On the Microsoft Investor Relations website, y...",1,9,"[-0.022940216585993767, -0.008343684487044811,..."
2,3,MSFT,23,1,GAAP. They are included as additional clarifyi...,1,17,"[-0.01130777969956398, -0.0038822712376713753,..."


In [20]:

# Create a search index
index_client = SearchIndexClient(
    endpoint=service_endpoint, credential=credential)
fields = [
    SimpleField(name="Id", type=SearchFieldDataType.String, key=True, sortable=True, filterable=True, facetable=True),
    SearchableField(name="Ticker", type=SearchFieldDataType.String, filterable=True),
    SearchableField(name="Year", type=SearchFieldDataType.String, filterable=True),
    SearchableField(name="Quarter", type=SearchFieldDataType.String, filterable=True),
    SearchableField(name="Chunk", type=SearchFieldDataType.String, searchable=True),
    SearchableField(name="PageNumber", type=SearchFieldDataType.String, filterable=True),
    SearchableField(name="LineNumber", type=SearchFieldDataType.String, filterable=True),
    
    SearchField(name="Embedding", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                searchable=True, vector_search_dimensions=1536, vector_search_configuration="my-vector-config"),
]

vector_search = VectorSearch(
    algorithm_configurations=[
        HnswVectorSearchAlgorithmConfiguration(
            name="my-vector-config",
            kind="hnsw",
            parameters={
                "m": 4,
                "efConstruction": 400,
                "efSearch": 500,
                "metric": "cosine"
            }
        )
    ]
)

semantic_config = SemanticConfiguration(
    name="my-semantic-config",
    prioritized_fields=PrioritizedFields(
        title_field=SemanticField(field_name="Ticker"),
        prioritized_content_fields=[SemanticField(field_name="Chunk")]
    )
)

# Create the semantic settings with the configuration
semantic_settings = SemanticSettings(configurations=[semantic_config])

# Create the search index with the semantic settings
index = SearchIndex(name=index_name, fields=fields,
                    vector_search=vector_search, semantic_settings=semantic_settings)
result = index_client.create_or_update_index(index)
print(f' {result.name} created')

 rag_prop_j_3 created


In [31]:

## Upload data to Index
def batch_append_payload(df, search_client):
    """append payload for batch insertion (note: max 1000 rows per insertion) of embeddings to Cognitive Search"""
    value_list = []
    for index, row in df.iterrows():
        value_list.append(
            {
                "Id": str(index),
                "Ticker": row["Ticker"],
                "Year": str(row["Year"]),
                "Quarter": str(row["Quarter"]),
                "Chunk": row["Chunk"],
                "PageNumber": str(row["PageNumber"]),
                "LineNumber": str(row["LineNumber"]),
                "Embedding": literal_eval(row['Embedding']),
            }
        )
        
#         print(len(value_list))
        
        if len(value_list) >= 1000:
            result = search_client.upload_documents(value_list)
            print(f"Uploaded {len(value_list)} payload")
            value_list = []
    result = search_client.upload_documents(value_list)
    print(f"Uploaded {len(value_list)} payload")
    
            
            
#     print('payload of size {}'.format(len(value_list)))

    return value_list


search_client = SearchClient(endpoint=service_endpoint, index_name=index_name, credential=credential)
payload = batch_append_payload(df_chunks_embedding, search_client)
 
# print(f"Uploaded {len(payload)} payload") 


Uploaded 442 payload


# Search Types 1: Pure Vector Search

In [51]:
# Pure Vector Search
query = "Microsoft earnings call for year 2022 for Quarter 2"  
  
search_client = SearchClient(service_endpoint, index_name, credential=credential)
vector = Vector(value=createEmbeddings(query), k=2, fields="Embedding")
  
results = search_client.search(  
    search_text=None,  
    vectors=[vector],
#     select=["Ticker", "Quarter", "Year"],
)

# results
  
for result in results: 
    print(result['Ticker'])
    print(result['Quarter'])
    print(result['Year'])
    print(result['Chunk'])
    break

MSFT
2
23
Microsoft FY23 Second Quarter Earnings Conference Call Brett Iversen, Satya Nadella, Amy Hood Tuesday, January 24, 2023 BRETT IVERSEN: Good afternoon and thank you for joining us today. On the call with me are Satya Nadella, chairman and chief executive officer, Amy Hood, chief financial officer, Alice Jolla, chief accounting officer, and Keith Dolliver, deputy general counsel. On the Microsoft Investor Relations website, you can find our earnings press release and financial summary slide deck, which is intended to 


# Search Types 2: Pure Filter

In [52]:
results = search_client.search(  
    search_text=None,  
    filter="(Ticker eq 'MSFT') and (Year eq '23') and (Quarter eq '1') ",
)  

for result in results:
    print(f"Ticker: {result['Ticker']}")
    print(f"Quarter: {result['Quarter']}") 
    print(f"Year: {result['Year']}") 
    print(result['Chunk'])
    print()
    break


Ticker: MSFT
Quarter: 1
Year: 23
Microsoft FY23 First Quarter Earnings Conference Call Brett Iversen, Satya Nadella, Amy Hood Tuesday, October 25, 2022 BRETT IVERSEN: Good afternoon and thank you for joining us today. On the call with me are Satya Nadella, chairman and chief executive officer, Amy Hood, chief financial officer, Alice Jolla, chief accounting officer, and Keith Dolliver, deputy general counsel. On the Microsoft Investor Relations website, you can find our earnings press release and financial summary slide deck, which is intended to 



# Search Types 3: Vector Search with filters

In [53]:
# Pure Vector Search with Filter
query = "What are the KPIs?"  
  
search_client = SearchClient(service_endpoint, index_name, credential=credential)  
vector = Vector(value=createEmbeddings(query), k=5, fields="Embedding")  

results = search_client.search(  
    search_text=None,  
    vectors=[vector],
    filter="(Ticker eq 'MSFT') and (Year eq '23') and (Quarter eq '1') ",
#     select=["Ticker", "Quarter", "Year"],
)  
  
for result in results:
    print(f"Ticker: {result['Ticker']}")
    print(f"Quarter: {result['Quarter']}") 
    print(f"Year: {result['Year']}") 
    print(result['Chunk'])
    print()

    break

Ticker: MSFT
Quarter: 1
Year: 23
you're still seeing digitization. This is still the tailwind that helps customers solve problems. This is still the way to build growth and leverage in your business. And yet, you still want to optimize your workloads. You still want to run them the most efficiently so that you can then make room for new workload growth. We saw that across all segments. If there was one segment where I may have seen it a bit more, I would say, in the small or mid-sized segment of the market, that tends to be more through partner. We rely on partners to help customers do those same optimizations and prepare workloads. But it is that one point I know that people are focused on. 



# Search Types 4: Hybrid Search with filters

In [54]:
# Pure Vector Search with Filter
query = "What are the KPIs?"  
  
search_client = SearchClient(service_endpoint, index_name, credential=credential)  
vector = Vector(value=createEmbeddings(query), k=5, fields="Embedding")  

results = search_client.search(  
    search_text=query,  
    vectors=[vector],
    filter="(Ticker eq 'MSFT') and (Year eq '23') and (Quarter eq '1') ",
#     select=["Ticker", "Quarter", "Year"],
    top = 3
)  
  
for result in results:
    print(f"Ticker: {result['Ticker']}")
    print(f"Quarter: {result['Quarter']}") 
    print(f"Year: {result['Year']}") 
    print(result['Chunk'])
    print()
    break

Ticker: MSFT
Quarter: 1
Year: 23
AMY HOOD: Thanks, Keith, and I do appreciate you asking about that one point, because I do know it is a point of focus every quarter. And what I would say is there is some inherent volatility to that number. A point here or there, and you've heard me say it when we've been a point better, and you've heard me say it when we've been a point worse. And I want to focus mostly on what and how we see the number, which is that it is still a very large growth rate with growth across all segments and with growth across all geos. That was, to the question, generally in line with where we expected. And what we did see through the quarter is a real focus both by customers, but also by our sales and customer success teams on 

